### 导入基本的类库

In [1]:
# 风险及免责提示：该策略由聚宽用户在聚宽社区分享，仅供学习交流使用。
# 原文一般包含策略说明，如有疑问请到原文和作者交流讨论。
# 原文网址：https://www.joinquant.com/view/community/detail/26816
# 标题：ETF资源收集整合（附代码与excel表格）

import numpy as np
import pandas as pd
from jqdata import *

import requests
import re
from bs4 import BeautifulSoup

### 从聚宽获取ETF信息

聚宽的信息还是比较全的。但是有些属性不全，比如想要知道哪些ETF可以T+0，哪些是跨境ETF，管理机构是谁。tushare上的免费数据可以拿，但要积分。所以打算结合站外数据做一下填充。

In [2]:
trade_date = get_trade_days(end_date=datetime.datetime.now(), count=10)
# 获取聚宽ETF数据
funds = get_all_securities(['fund'], date=trade_date[-1])  # 所有基金
etfs = funds[funds['type'] == 'etf']  # 挑出ETF
etfs.tail()

,display_name,name,start_date,end_date,type
515950.XSHG,医药50FG,YY50FG,2020-04-01,2200-01-01,etf
515980.XSHG,人工智能,RGZN,2020-02-10,2200-01-01,etf
515990.XSHG,添富国企,TFGQ,2020-01-15,2200-01-01,etf
518800.XSHG,黄金基金,GTHJ,2013-07-29,2200-01-01,etf
518880.XSHG,黄金ETF,HJETF,2013-07-29,2200-01-01,etf


In [3]:
# 格式整理
df_etfs = pd.DataFrame(index=etfs.index)
codes = [code.split('.')[0] for code in etfs.index.values]
df_etfs['代码'] = codes
df_etfs['全称'] = etfs.display_name
df_etfs['简称'] = etfs.name
df_etfs['开始时间'] = etfs.start_date
df_etfs['结束时间'] = etfs.end_date
df_etfs.reset_index(inplace=True, drop=True)
df_etfs.tail()

,代码,全称,简称,开始时间,结束时间
282,515950,医药50FG,YY50FG,2020-04-01,2200-01-01
283,515980,人工智能,RGZN,2020-02-10,2200-01-01
284,515990,添富国企,TFGQ,2020-01-15,2200-01-01
285,518800,黄金基金,GTHJ,2013-07-29,2200-01-01
286,518880,黄金ETF,HJETF,2013-07-29,2200-01-01


### 从站外获取ETF数据

In [5]:
# html 内容请求
def get_html(code):
    # url中的参数是个ETF的代码
    link = 'http://.../'+code+'.html?spm=search'  # 为了成为一个有责任心的爬虫者，这里暂不公开网站地址啦
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6'} 
    r = requests.get(link, headers= headers)
    r.encoding='utf-8'
    return r.text

In [21]:
# 获取特征内容
def get_content(code):
    text = get_html(code)
    soup = BeautifulSoup(text,"lxml")
    div = soup.find("div", class_="infoOfFund")
    td = div.find_all('td')

    fund_dict = dict()
    fund_dict['代码'] = code
    for t in td:
        text = t.text
        key_value = re.split('[:：|]',text.strip())
        fund_dict[key_value[0]] = key_value[1]
        if len(key_value) > 2:
            fund_dict[key_value[2]] = key_value[3]

    return fund_dict

In [22]:
# 测试爬到的数据
get_content('515950')

{'代码': '515950',
 '基金类型': 'ETF-场内',
 '基金规模': '2.88亿元（2020-03-25）',
 '基金经理': '王乐乐',
 '成 立 日': '2020-03-16',
 '管 理 人': '富国基金',
 '基金评级': '暂无评级',
 '跟踪标的': '中证医药50指数 ',
 ' 跟踪误差': '--'}

In [23]:
# 开始将所有的数据爬一遍，并组合成DataFrame格式
list_f = []
for code in df_etfs['代码']:
    try:
        contect_dict = get_content(code)
        list_f.append(contect_dict)
    except:
        print(code+'的内容查不到')
df_f = pd.DataFrame(list_f)
df_f.head()

,跟踪误差,代码,基金类型,基金经理,基金规模,基金评级,成 立 日,管 理 人,跟踪标的
0,NaN,159001,货币型,石大怿等,2.07亿元（2020-03-31）,暂无评级,2013-03-29,易方达基金,NaN
1,NaN,159003,货币型,许强,1.65亿元（2020-03-31）,暂无评级,2013-05-17,招商基金,NaN
2,NaN,159005,货币型,温开强,0.46亿元（2020-03-31）,暂无评级,2014-12-23,汇添富基金,NaN
3,--,159801,ETF-场内,罗国庆,46.06亿元（2020-03-31）,暂无评级,2020-01-20,广发基金,国证半导体芯片指数
4,--,159804,ETF-场内,李康,3.43亿元（2020-03-24）,暂无评级,2020-03-04,国寿安保基金,国证创业板中盘精选88指数


### 内容合并

最后的步骤就简单，将聚宽的数据和爬到的数据整合到一起，然后导出。将excel收到自己的备用百宝箱里，常用常新。今后有需要的其它属性后再更新。

In [46]:
e_f = df_etfs.set_index("代码")
f_f = df_f.set_index("代码")
f_f['名称'] = e_f['全称']
f_f.head()

,跟踪误差,基金类型,基金经理,基金规模,基金评级,成 立 日,管 理 人,跟踪标的,名称
代码,,,,,,,,,
159001,NaN,货币型,石大怿等,2.07亿元（2020-03-31）,暂无评级,2013-03-29,易方达基金,NaN,保证金
159003,NaN,货币型,许强,1.65亿元（2020-03-31）,暂无评级,2013-05-17,招商基金,NaN,招商快线
159005,NaN,货币型,温开强,0.46亿元（2020-03-31）,暂无评级,2014-12-23,汇添富基金,NaN,添富快钱
159801,--,ETF-场内,罗国庆,46.06亿元（2020-03-31）,暂无评级,2020-01-20,广发基金,国证半导体芯片指数,芯片基金
159804,--,ETF-场内,李康,3.43亿元（2020-03-24）,暂无评级,2020-03-04,国寿安保基金,国证创业板中盘精选88指数,创精选88


In [48]:
# 最基础类型数据导出
f_f.to_excel('ETFS.xlsx')

___

#### 针对宽友的要求，后续添加的数据列（持续更新)

In [54]:
trade_date = get_trade_days(end_date=datetime.datetime.now(), count=10)

In [56]:
# 增加当日单位净值数据 2020-4-14
codes = [normalize_code(code) for code in e_f['代码']]
unit_net_values = get_extras('unit_net_value', 
                             codes, 
                             end_date=trade_date[-2], 
                             count=1,
                             df=False)
df_unit_net = pd.DataFrame(unit_net_values, index=['unit_net_value']).T
df_unit_net.tail()  # 净值查看

,unit_net_value
515950.XSHG,1.1117
515980.XSHG,1.0057
515990.XSHG,0.9125
518800.XSHG,3.6335
518880.XSHG,3.6665


In [64]:
# 数据合并
indexs = [normalize_code(code) for code in e_f['代码']]
df_old = e_f
df_old['code'] = indexs
df_old.set_index('code', inplace=True)
df_old['基金净值'] = df_unit_net['unit_net_value']
df_new = df_old.reset_index(drop=False)
df_new.tail()

,code,代码,全称,简称,开始时间,结束时间,单位净值,基金类型,基金经理,基金规模,基金评级,管理人,基金净值
300,515950.XSHG,515950,医药50FG,YY50FG,2020-04-01,2200-01-01,NaN,ETF-场内,王乐乐,2.88亿元（2020-03-25）,暂无评级,富国基金,1.1117
301,515980.XSHG,515980,人工智能,RGZN,2020-02-10,2200-01-01,NaN,ETF-场内,张娅,3.00亿元（2020-02-03）,暂无评级,华富基金,1.0057
302,515990.XSHG,515990,添富国企,TFGQ,2020-01-15,2200-01-01,NaN,ETF-场内,吴振翔,36.97亿元（2020-01-08）,暂无评级,汇添富基金,0.9125
303,518800.XSHG,518800,黄金基金,GTHJ,2013-07-29,2200-01-01,NaN,ETF-场内,艾小军,10.60亿元（2019-12-31）,暂无评级,国泰基金,3.6335
304,518880.XSHG,518880,黄金ETF,HJETF,2013-07-29,2200-01-01,NaN,ETF-场内,许之彦,69.95亿元（2019-12-31）,暂无评级,华安基金,3.6665


In [66]:
# 包含净值列数据的导出
df_new.to_excel('ETFS_NEW.xlsx')